In [1]:
from CNN import CNN
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils import set_random_seed, image_dataset_from_directory
import os
import tensorflow as tf
from utils import prepare_dataset

In [2]:
SEED = 42
DATASET_PATH = "../../../dataset/"
EPOCHS = 150
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE
INPUT_SHAPE = (224, 224, 3, )

In [3]:
model_path = "../CNN_training/keras_tuner/"
cnn = CNN(model_path)
cnn.cnn.summary()

INFO:tensorflow:Reloading Oracle from existing project ../CNN_training/keras_tuner/tuned_model\oracle.json
INFO:tensorflow:Reloading Tuner from ../CNN_training/keras_tuner/tuned_model\tuner0.json
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 44, 44, 32)       0         
 2D)                                                             
                                                                 
 conv2d_15 (Conv2D)          (None, 42, 42, 256)       73984     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 21, 21, 256)      0         
 2D)                                                             
                                                        

In [5]:
# Load the training dataset
print("Loading train dataset...")
train_dataset = image_dataset_from_directory(
    directory=os.path.join(DATASET_PATH, "train"),
    label_mode="categorical",
    validation_split=None,
    image_size=(224, 224),
    batch_size=None,
    seed=SEED
    )
print("Train dataset loaded!")
print("Labels in the dataset: ", train_dataset.class_names)

# Load the validation dataset
print("Loading validation dataset...")
val_dataset = image_dataset_from_directory(
    directory=os.path.join(DATASET_PATH, "valid"),
    label_mode="categorical",
    validation_split=None,
    image_size=(224, 224),
    batch_size=None,
    seed=SEED
    )
print("Validation dataset loaded!")

train_ds = prepare_dataset(train_dataset, SEED, AUTOTUNE, BATCH_SIZE, augment=False)
val_ds = prepare_dataset(val_dataset, SEED, AUTOTUNE, BATCH_SIZE)

len(train_ds), len(val_ds)

Loading train dataset...
Found 11280 files belonging to 2 classes.
Train dataset loaded!
Labels in the dataset:  ['savory', 'unsavory']
Loading validation dataset...
Found 600 files belonging to 2 classes.
Validation dataset loaded!


(705, 38)

In [ ]:
early_stopping = EarlyStopping(monitor="val_loss", patience=12, min_delta=0.0001, restore_best_weights=True, verbose=1)
lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, verbose=1)
callbacks = [early_stopping, lr_scheduler]

In [ ]:
cnn.cnn.fit(train_ds, epochs=50, callbacks=callbacks, validation_data=val_ds)

In [ ]:
cnn.cnn.save("./trained_cnn")